In [1]:
import pandas as pd
import numpy as np

import scripts.word2vec as w2v

import hdbscan
import umap

In [2]:
politicians_quotes = pd.read_csv('politician_quote_data/quotes-2019-politicians.csv.gz', compression='gzip')[['quoteID','quotation']]

In [3]:
politicians_quotes

,quoteID,quotation
0,2019-04-18-000603,a contemporary architectural gesture.
1,2019-02-14-001346,a disgrace on so many levels
2,2019-08-15-047354,It took us 70 years to reach $2 trillion econo...
3,2019-04-03-003342,acting as private citizens.
4,2019-05-04-027585,It was said you (Mayawati) will be made the pr...
...,...,...
437167,2019-01-22-047525,International students are a vital part of Vic...
437168,2019-08-07-044986,is intended to insure against downside risks f...
437169,2019-10-30-052703,Isn't that the very definition of a quid pro quo?
437170,2019-12-14-021964,issues that matter to both of our countries


In [4]:
# Extend dataframe with the quotes vectors

w2v.extend_dataframe(politicians_quotes, 'quotation', 'quotation_vector')

### Loading Word2Vec glove-wiki-gigaword-50 pre trained model ###
### Successfully loaded Word2Vec glove-wiki-gigaword-50 pre trained model ###


C:\Users\Andi\anaconda3\envs\ada-p\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Andi\anaconda3\envs\ada-p\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [5]:
#embedding contains nans?
politicians_quotes.isna().sum()

quoteID                0
quotation              0
quotation_vector    3579
dtype: int64

In [6]:
#remove nans
politicians_quotes = politicians_quotes[politicians_quotes['quotation_vector'].isna().apply(lambda x: not x)]

In [7]:
#save quote data? 
#politicians_quotes.to_csv(path_or_buf="embedded_data/allyears.csv.gz", compression='gzip')

In [8]:
doc_array_for_umap = np.stack(politicians_quotes['quotation_vector'].to_numpy())

In [9]:
umap_args = {'n_neighbors': 15,
                         'n_components': 5,
                         'metric': 'cosine'}
hdbscan_args = {'min_cluster_size': 15,
                            'metric': 'euclidean',
                            'cluster_selection_method': 'eom'}

In [10]:
umap_model = umap.UMAP(**umap_args).fit(doc_array_for_umap)

In [13]:
cluster = hdbscan.HDBSCAN(**hdbscan_args, core_dist_n_jobs=1).fit(umap_model.embedding_)

In [14]:
cluster.labels_

array([ -1,  -1, 985, ..., 139, 826,  -1], dtype=int64)

In [15]:
politicians_quotes['cluster'] = cluster.labels_

C:\Users\Andi\AppData\Local\Temp/ipykernel_5336/746385989.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  politicians_quotes['cluster'] = cluster.labels_


In [17]:
politicians_quotes

,quoteID,quotation,quotation_vector,cluster
0,2019-04-18-000603,a contemporary architectural gesture.,"[-0.29759407, 1.4591777, -2.2330098, -0.501039...",-1
1,2019-02-14-001346,a disgrace on so many levels,"[-0.72491825, 0.54614925, -0.03522542, -1.5560...",-1
2,2019-08-15-047354,It took us 70 years to reach $2 trillion econo...,"[0.21631472, 0.18373148, 1.2528119, -0.1479444...",985
3,2019-04-03-003342,acting as private citizens.,"[0.36425605, 0.046690255, -0.3002879, -0.68249...",-1
4,2019-05-04-027585,It was said you (Mayawati) will be made the pr...,"[-0.18833013, 0.31297463, -0.15920538, 0.64997...",-1
...,...,...,...,...
437167,2019-01-22-047525,International students are a vital part of Vic...,"[-0.41933396, 0.77122176, -0.5849735, -1.14137...",862
437168,2019-08-07-044986,is intended to insure against downside risks f...,"[0.24927647, -0.19867755, 0.697066, -0.4656627...",981
437169,2019-10-30-052703,Isn't that the very definition of a quid pro quo?,"[-1.8786557, 0.42048725, 0.0072864275, 0.43749...",139
437170,2019-12-14-021964,issues that matter to both of our countries,"[0.19947365, 0.43275028, -0.4803699, 0.3564474...",826


In [18]:
politicians_quotes.query("cluster != -1")

,quoteID,quotation,quotation_vector,cluster
2,2019-08-15-047354,It took us 70 years to reach $2 trillion econo...,"[0.21631472, 0.18373148, 1.2528119, -0.1479444...",985
14,2019-12-10-005732,And that is the message I've had from her both...,"[0.97310185, 0.13664214, 0.34558702, -0.642887...",756
32,2019-01-25-063513,"Likewise, India could also partner with South ...","[0.5674079, -0.44662404, 0.5392596, 0.7250929,...",973
35,2019-08-23-053146,Low inflation seems to be the problem of this ...,"[-0.5747592, 0.25580204, 0.8693842, -1.1770443...",981
42,2019-08-06-010685,But there is always room for talks and negotia...,"[1.3722614, 0.71909064, -0.7578106, 0.36259907...",674
...,...,...,...,...
437162,2019-08-04-020981,"In July, Andrew Scheer travelled to Whitehorse...","[0.9424799, 1.0193243, -0.004788083, -0.374829...",692
437167,2019-01-22-047525,International students are a vital part of Vic...,"[-0.41933396, 0.77122176, -0.5849735, -1.14137...",862
437168,2019-08-07-044986,is intended to insure against downside risks f...,"[0.24927647, -0.19867755, 0.697066, -0.4656627...",981
437169,2019-10-30-052703,Isn't that the very definition of a quid pro quo?,"[-1.8786557, 0.42048725, 0.0072864275, 0.43749...",139


In [ ]:
#calculate cluster avg = topic vector
topic_vectors = politicians_quotes[''].groupby('cluster')

In [25]:
politicians_quotes.query("cluster == 8")

,quoteID,quotation,quotation_vector,cluster
770,2019-07-02-079518,"This is the hope for Hong Kong,","[0.50341964, 1.1244245, 0.12550178, -0.3417700...",8
32019,2019-01-24-120859,"We put our hope in him,","[0.50341964, 1.1244245, 0.12550178, -0.3417700...",8
56689,2019-12-11-032303,"I hope they are for it,","[0.50341964, 1.1244245, 0.12550178, -0.3417700...",8
58111,2019-01-28-088503,they don't have a hope in God.,"[0.50341964, 1.1244245, 0.12550178, -0.3417700...",8
66753,2019-01-18-026522,"Help us, Robert Mueller. You're our only hope.","[0.50341964, 1.1244245, 0.12550178, -0.3417700...",8
80995,2019-09-12-129070,"You hope not, you hope not,","[0.50341964, 1.1244245, 0.12550178, -0.3417700...",8
87343,2019-07-16-029817,I hope we will make it together with Russia,"[0.50341964, 1.1244245, 0.12550178, -0.3417700...",8
90631,2019-07-23-040397,Maybe they will make me one. I hope so.,"[0.50341964, 1.1244245, 0.12550178, -0.3417700...",8
166501,2019-09-06-097229,This bill is the torch of hope.,"[0.086004466, 1.7274868, 0.36342892, 0.3240472...",8
172209,2019-11-06-124417,"Westy gave us a glimmer of hope,","[0.8726438, 1.3489392, 0.14481518, -0.2759317,...",8
